In [1]:
import pandas as pd 

In [2]:
#  处理个人出行表 和 有私家车家庭信息去重表
DL_trip_origin = pd.read_csv("data/original/个人出行.csv")[["Family_NO", "Person_NO", "Travel_Purpose", "MainTransport",
                                         "Departure_Time", "Arrival_Time", "出行时长", "distance", "Departure_Itude",
                                         "Destination_Itude", "SumCost"]]

DL_trip_origin.rename(columns={'出行时长':'trip_time'},inplace=True)

DL_trip_origin["Departure_longitude"] = DL_trip_origin["Departure_Itude"].str.split(',', expand=True)[0]
DL_trip_origin["Departure_latitude"] = DL_trip_origin["Departure_Itude"].str.split(',', expand=True)[1]

DL_trip_origin["Destination_longitude"] = DL_trip_origin["Destination_Itude"].str.split(',', expand=True)[0]
DL_trip_origin["Destination_latitude"] = DL_trip_origin["Destination_Itude"].str.split(',', expand=True)[1]

DL_trip_origin.drop(["Departure_Itude", "Destination_Itude"],axis=1,inplace=True)

# # 私家车信息加入到个人出行表中

family_car_info = pd.read_csv("data/original/有私家车家庭信息去重.csv")[["Family_NO"]]
family_car_info.insert(1,value = 1,column="Cars_Count")
DL_trip_final = pd.merge(DL_trip_origin,family_car_info,how='left')
DL_trip_final['Cars_Count']=DL_trip_final['Cars_Count'].fillna(0)
DL_trip_final['Cars_Count']=DL_trip_final['Cars_Count'].astype('int')

In [3]:
# 处理家庭信息表 和 家庭成员信息表
DL_family_info = pd.read_csv("data/original/家庭信息.csv")[["Family_NO",  'Total_IN', 'Area', 'Nature', 'Members_Count']]

DL_family_member_info = pd.read_csv("data/original/家庭成员信息.csv")[
    ["Family_NO", "Person_NO", 'Sex', 'Age', 'Education', 'Career_Code', 'Is_Driver', 'Monday', 'Tuesday', 'Wednesday',
     'Thursday', 'Friday', 'Saturday', 'Sunday']]

# 合并 DL_family_member_info 和 DL_family_info
DL_family_info_final = pd.merge(DL_family_member_info, DL_family_info,on="Family_NO" , how="left")

In [4]:
DL_data_final = pd.merge(DL_trip_final, DL_family_info_final,on=["Family_NO",'Person_NO'] , how="inner")
DL_data_final.reset_index(inplace=True)
DL_data_final.to_csv("data/DL_data_final.csv")
print(DL_data_final.index)
print(DL_data_final.columns)

RangeIndex(start=0, stop=72851, step=1)
Index(['index', 'Family_NO', 'Person_NO', 'Travel_Purpose', 'MainTransport',
       'Departure_Time', 'Arrival_Time', 'trip_time', 'distance', 'SumCost',
       'Departure_longitude', 'Departure_latitude', 'Destination_longitude',
       'Destination_latitude', 'Cars_Count', 'Sex', 'Age', 'Education',
       'Career_Code', 'Is_Driver', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday', 'Friday', 'Saturday', 'Sunday', 'Total_IN', 'Area',
       'Nature', 'Members_Count'],
      dtype='object')
